# Rekko Sandbox

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os
import sys
import re
import logging

from collections import Counter

from sklearn.feature_extraction.text import CountVectorizer

from time import time

from pyspark.sql import SparkSession

In [3]:
parent_dir = os.path.split(os.getcwd())[0]
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [4]:
parent_dir

'/Users/roseaysina/code/sponge-bob-magic'

In [5]:
from sponge_bob_magic.data_preparator.data_preparator import DataPreparator
from sponge_bob_magic.scenarios.popular_scenario import PopularScenario
from sponge_bob_magic.models.popular_recomennder import PopularRecommender
from sponge_bob_magic.metrics.metrics import Metrics

In [6]:
# Отображение максимальной ширины колонок в pandas датафреймах
pd.options.display.max_colwidth = -1

In [7]:
spark_memory = "2g"
spark_cores = "*"

spark = (
    SparkSession
    .builder
    .config('spark.driver.memory', spark_memory)
    .master(f'local[{spark_cores}]')
    .enableHiveSupport()
    .getOrCreate()
)

In [8]:
spark_logger = logging.getLogger('py4j')
spark_logger.setLevel(logging.WARN)

In [9]:
logger = logging.getLogger()
formatter = logging.Formatter('%(asctime)s, %(name)s, %(levelname)s: %(message)s',
                              datefmt='%d-%b-%y %H:%M:%S')
hdlr = logging.StreamHandler()
hdlr.setFormatter(formatter)
logger.addHandler(hdlr)
logger.setLevel(logging.DEBUG)

logging.debug("test")

01-Oct-19 15:04:08, root, DEBUG: test


## Основные данные и пути

In [10]:
path_data = '/Users/roseaysina/code/sponge-bob-magic/data/rekko_sand_rekko'
path_transactions = os.path.join(path_data, 'transactions.csv')
path_checkpoints = '/Users/roseaysina/code/sponge-bob-magic/data/checkpoints'

In [11]:
%%time

transactions = pd.read_csv(
    path_transactions,
    dtype={
        'element_uid': np.uint16,
        'user_uid': np.uint32,
        'consumption_mode': 'category',
        'ts': np.float64,
        'watched_time': np.uint64,
        'device_type': np.uint8,
        'device_manufacturer': np.uint8
    }
)

CPU times: user 3.56 s, sys: 353 ms, total: 3.92 s
Wall time: 3.76 s


In [12]:
transactions.head()

,element_uid,user_uid,consumption_mode,ts,watched_time,device_type,device_manufacturer
0,3336,5177,S,4.430518e+07,4282,0,50
1,481,593316,S,4.430518e+07,2989,0,11
2,4128,262355,S,4.430518e+07,833,0,50
3,6272,74296,S,4.430518e+07,2530,0,99
4,5543,340623,P,4.430518e+07,6282,0,50


In [13]:
transactions.nunique()

element_uid            8296   
user_uid               499663 
consumption_mode       3      
ts                     9634273
watched_time           124958 
device_type            7      
device_manufacturer    100    
dtype: int64

## Дата препаратор

In [14]:
dp = DataPreparator(spark)

In [15]:
df = dp.transform_log(
    path_transactions,
    format_type='csv',
    columns_names={
        'user_id': 'user_uid', 
        'item_id': 'element_uid',
        'relevance': 'watched_time'
    },
    date_format=None,
    header=True
)

In [16]:
df.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- item_id: string (nullable = true)
 |-- relevance: float (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- context: string (nullable = false)



In [17]:
df.show(3)

+-------+-------+---------+-------------------+----------+
|user_id|item_id|relevance|          timestamp|   context|
+-------+-------+---------+-------------------+----------+
|   5177|   3336|   4282.0|1999-05-01 00:00:00|no_context|
| 593316|    481|   2989.0|1999-05-01 00:00:00|no_context|
| 262355|   4128|    833.0|1999-05-01 00:00:00|no_context|
+-------+-------+---------+-------------------+----------+
only showing top 3 rows



## Популярный рекоммендер

In [18]:
spark.sparkContext.setCheckpointDir(path_checkpoints)

In [19]:
pr = PopularRecommender(spark, alpha=0, beta=0)

In [20]:
%%time

pr.fit(
    log=df,
    user_features=None,
    item_features=None,
    path=path_checkpoints
)

CPU times: user 9.77 ms, sys: 4.23 ms, total: 14 ms
Wall time: 13.6 s


In [21]:
%%time

recs = pr.predict(
    k=10,
    users=None,
    items=None,
    context='no_context',
    log=df,
    user_features=None,
    item_features=None,
    to_filter_seen_items=True,
    path=None
)

01-Oct-19 15:04:39, root, DEBUG: Проверка датафреймов
01-Oct-19 15:04:47, root, DEBUG: Выделение дефолтных юзеров
01-Oct-19 15:04:47, root, DEBUG: Выделение дефолтных айтемов
01-Oct-19 15:04:51, root, DEBUG: Выделение контекста
01-Oct-19 15:04:51, root, DEBUG: Подсчет количества айтемов
01-Oct-19 15:04:52, root, DEBUG: Подсчет популярности
01-Oct-19 15:04:52, root, DEBUG: Удаление ненужных айтемов
01-Oct-19 15:04:52, root, DEBUG: Заполнение нуллов
01-Oct-19 15:04:58, root, DEBUG: k для выделения топа: 20.0
01-Oct-19 15:05:07, root, DEBUG: Количество айтемов после фильтрации: 30
01-Oct-19 15:05:07, root, DEBUG: Кросс-джойн юзеров на айтемы
01-Oct-19 15:05:29, root, DEBUG: Длина recs: 14989890
01-Oct-19 15:05:29, root, DEBUG: Удаление виденных айтемов
01-Oct-19 15:06:07, root, DEBUG: Берем топ-к
01-Oct-19 15:06:45, root, DEBUG: Заменяем отрицательные рейтинги на 0


CPU times: user 67.9 ms, sys: 22.1 ms, total: 90 ms
Wall time: 2min 53s


In [22]:
recs.show(5)

+-------+-------+----------+--------------------+
|item_id|user_id|   context|           relevance|
+-------+-------+----------+--------------------+
|   2714| 100140|no_context|0.007084923258417598|
|    747| 100140|no_context|0.006354031292297469|
|   6127| 100140|no_context|0.005409927935379527|
|   3916| 100140|no_context|0.005002897434950823|
|   8771| 100140|no_context|0.004923772779708249|
+-------+-------+----------+--------------------+
only showing top 5 rows



In [23]:
metric = Metrics.hit_rate_at_k(recs, df, k=10)
metric

0.0002561726603730915

## Популярный сценарий

In [24]:
df_short = df.limit(1000)
df_short.checkpoint()

DataFrame[user_id: string, item_id: string, relevance: float, timestamp: timestamp, context: string]

In [25]:
popular_scenario = PopularScenario(spark)
popular_params_grid = {'alpha': (0, 100), 'beta': (0, 100)}

In [26]:
%%time

best_params = popular_scenario.research(
    popular_params_grid,
    df,
    users=None, items=None,
    user_features=None,
    item_features=None,
    test_start=None,
    test_size=0.35,
    k=10, context='no_context',
    to_filter_seen_items=True,
    n_trials=1, n_jobs=1,
    how_to_split='randomly'
)

01-Oct-19 15:09:22, root, DEBUG: Деление на трейн и тест
01-Oct-19 15:09:32, root, DEBUG: Размер трейна:      682
01-Oct-19 15:09:35, root, DEBUG: Размер теста_инпут: 682
01-Oct-19 15:09:38, root, DEBUG: Размер теста:       130
01-Oct-19 15:09:59, root, DEBUG: Начало оптимизации параметров
01-Oct-19 15:09:59, root, DEBUG: Предикт модели в оптимизации
01-Oct-19 15:09:59, root, DEBUG: Проверка датафреймов
01-Oct-19 15:10:12, root, DEBUG: Выделение дефолтных юзеров
01-Oct-19 15:10:12, root, DEBUG: Выделение дефолтных айтемов
01-Oct-19 15:10:12, root, DEBUG: Выделение контекста
01-Oct-19 15:10:12, root, DEBUG: Подсчет количества айтемов
01-Oct-19 15:10:16, root, DEBUG: Подсчет популярности
01-Oct-19 15:10:16, root, DEBUG: Удаление ненужных айтемов
01-Oct-19 15:10:16, root, DEBUG: Заполнение нуллов
01-Oct-19 15:10:20, root, DEBUG: k для выделения топа: 2.0
01-Oct-19 15:10:29, root, DEBUG: Количество айтемов после фильтрации: 12
01-Oct-19 15:10:29, root, DEBUG: Кросс-джойн юзеров на айтемы
0

CPU times: user 380 ms, sys: 127 ms, total: 508 ms
Wall time: 2min 1s


In [28]:
best_params

{'alpha': 58, 'beta': 32}

In [29]:
best_recs = popular_scenario.production(
    {'alpha': 0, 'beta': 0},
    df,
    users=None,
    items=None,
    user_features=None,
    item_features=None,
    k=10,
    context='no_context',
    to_filter_seen_items=True
)

01-Oct-19 15:13:32, root, DEBUG: Проверка датафреймов
01-Oct-19 15:13:40, root, DEBUG: Выделение дефолтных юзеров
01-Oct-19 15:13:40, root, DEBUG: Выделение дефолтных айтемов
01-Oct-19 15:13:43, root, DEBUG: Выделение контекста
01-Oct-19 15:13:43, root, DEBUG: Подсчет количества айтемов
01-Oct-19 15:13:49, root, DEBUG: Подсчет популярности
01-Oct-19 15:13:49, root, DEBUG: Удаление ненужных айтемов
01-Oct-19 15:13:49, root, DEBUG: Заполнение нуллов
01-Oct-19 15:13:56, root, DEBUG: k для выделения топа: 20.0
01-Oct-19 15:14:12, root, DEBUG: Количество айтемов после фильтрации: 30
01-Oct-19 15:14:12, root, DEBUG: Кросс-джойн юзеров на айтемы
01-Oct-19 15:14:41, root, DEBUG: Длина recs: 14989890
01-Oct-19 15:14:41, root, DEBUG: Удаление виденных айтемов
01-Oct-19 15:15:20, root, DEBUG: Берем топ-к
01-Oct-19 15:16:06, root, DEBUG: Заменяем отрицательные рейтинги на 0


In [30]:
best_recs

DataFrame[item_id: string, user_id: string, context: string, relevance: double]